<a href="https://colab.research.google.com/github/arsyidarmawan/RNN-Exercises/blob/main/02_RNN_Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>
<strong><center>Copyright 2019. Created by Jose Marcial Portilla.</center></strong>

# RNN Exercises
For these exercises we're using data from the Federal Reserve Economic Database (FRED) concerning Electricity and Gas Utilities Production from January 1992 to January 2019 (325 records).

Data source: https://fred.stlouisfed.org/series/IPG2211A2N

In the exercises below you'll be asked to do the following:
* Perform standard imports, load & plot the dataset (code provided)
* Prepare data for an LSTM model
* Define the LSTM model, loss and optimization functions
* Train the model
* Evaluate the model on test data
* OPTIONAL: Plot the results

<div class="alert alert-danger" style="margin: 10px"><strong>IMPORTANT NOTE!</strong> Make sure you don't run the cells directly above the example output shown, <br>otherwise you will end up writing over the example output!</div>

**NAMA : MUHAMAD ARSYI DARMAWAN**

**NPM : 183112706450268**

## Perform standard imports, load and plot the dataset
Run the cells below to load the libraries needed for this exercise and the Energy Production dataset, and to plot the data.

In [ ]:
# RUN THIS CELL
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

df = pd.read_csv('https://raw.githubusercontent.com/arsyidarmawan/RNN-Exercises/main/Energy_Production.csv')
df.dropna(inplace=True)
print(len(df))
df.head()

In [ ]:
# RUN THIS CELL
plt.figure(figsize=(12,4))
plt.title('Energy Production')
plt.ylabel('Sales (millions of dollars)')
plt.grid(True)
plt.autoscale(axis='x',tight=True)
plt.plot(df['IPG2211A2N'])
plt.show()

# Prepare the data
For the first set of exercises we'll
* divide the data into train and test sets
* normalize the training set
* prepare windowed seq/label tuples for an LSTM model

## 1. Divide the data into train and test sets
Working with a window_size of 12, divide the dataset into a sequence of 313 training records (including the window), and a test set of 12 records.

In [ ]:
# CODE HERE
# Extract values from the source .csv file
y = df['IPG2211A2N'].values.astype(float)

# Define a test size
test_size = 12
window_size = 12

# Create train and test sets
train_set = y[:-test_size]
test_set = y[-test_size:]

In [ ]:
# Run the code below to check your results:
print(f'Train: {len(train_set)}')
print(f'Test:  {len(test_set)}')

In [ ]:
# DON'T WRITE HERE

## 2. Normalize the training set
Feature scale the training set to fit within the range [-1,1].

In [ ]:
from sklearn.preprocessing import MinMaxScaler

#Instantiate a scaler with a feature range from -1 to 1
scaler = MinMaxScaler(feature_range=(-1, 1))
#Normalize the training set
train_norm = scaler.fit_transform(train_set.reshape(-1, 1))
train_norm.min()
train_norm.max()
train_norm.mean()

In [ ]:
# Run the code below to check your results:
print(f'First item, original: {train_set[0]}')
print(f'First item, scaled:  {train_norm[0]}')

In [ ]:
# DON'T WRITE HERE

## 3. Prepare data for LSTM
Prepare the list of windowed sequence/label tuples to be fed into an LSTM model.

In [ ]:
# RUN THIS CELL
train_norm = torch.FloatTensor(train_norm).view(-1)

def input_data(seq,ws):
    out = []
    L = len(seq)
    for i in range(L-ws):
        window = seq[i:i+ws]
        label = seq[i+ws:i+ws+1]
        out.append((window,label))
    return out

In [ ]:
# CODE HERE
train_data = input_data(train_norm,window_size)
len(train_data)

In [ ]:
# Run the code below to check your results:
print(f'Train_data: {len(train_data)}')  # should equal 301

In [ ]:
# DON'T WRITE HERE

## 4. Define the model
Design a model that has a (1,64) LSTM layer and a (64,1) fully-connected linear layer. Be sure to initialize $h_0$ and $c_0$, and return only the last predicted value.

In [ ]:
# CODE HERE
class LSTMnetwork(nn.Module):
    def __init__(self,input_size=1,hidden_size=64,output_size=1):
        super().__init__()
        self.hidden_size = hidden_size
        
        # Add an LSTM layer:
        self.lstm = nn.LSTM(input_size,hidden_size)
        
        # Add a fully-connected layer:
        self.linear = nn.Linear(hidden_size,output_size)
        
        # Initialize h0 and c0:
        self.hidden = (torch.zeros(1,1,self.hidden_size),
                       torch.zeros(1,1,self.hidden_size))

    def forward(self,seq):
        lstm_out, self.hidden = self.lstm(
            seq.view(len(seq),1,-1), self.hidden)
        pred = self.linear(lstm_out.view(len(seq),-1))
        return pred[-1] 


In [ ]:
# Run the code below to check your results:
torch.manual_seed(101)
model = LSTMnetwork()
model

In [ ]:
# DON'T WRITE HERE

## 5. Define loss and optimization functions
Define a loss function called "criterion" and an optimizer called "optimizer".<br>
You can use any functions you want, although we used MSELoss and Adam (learning rate of 0.001) respectively.

In [ ]:
torch.manual_seed(101)
model = LSTMnetwork()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model

def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    for item in params:
        print(f'{item:>6}')
    print(f'______\n{sum(params):>6}')
    
count_parameters(model)

In [ ]:
# DON'T WRITE HERE

### 6. Train the model
Don't worry about tracking loss values, displaying results, or validating the test set. Just train the model through 50 epochs. We'll evaluate the trained model in the next step.<br>
OPTIONAL: print something after each epoch to indicate training progress.

In [45]:
# CODE HERE

epochs = 50

import time
start_time = time.time()

for epoch in range(epochs):
    
    # extract the sequence & label from the training data
    for seq, y_train in train_data:
        
        # reset the parameters and hidden states
        optimizer.zero_grad()
        model.hidden = (torch.zeros(1,1,model.hidden_size),
                        torch.zeros(1,1,model.hidden_size))
        
        y_pred = model(seq)
        
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        
    # print training result
    print(f'Epoch: {epoch+1:2} Loss: {loss.item():10.8f}')
    
print(f'\nDuration: {time.time() - start_time:.0f} seconds')


Epoch:  1 Loss: 0.73148304
Epoch:  2 Loss: 0.32224393
Epoch:  3 Loss: 0.24065648
Epoch:  4 Loss: 0.22756016
Epoch:  5 Loss: 0.21153599
Epoch:  6 Loss: 0.19906579
Epoch:  7 Loss: 0.19067460
Epoch:  8 Loss: 0.18480708
Epoch:  9 Loss: 0.17962381
Epoch: 10 Loss: 0.17441957
Epoch: 11 Loss: 0.16898853
Epoch: 12 Loss: 0.16325688
Epoch: 13 Loss: 0.15732835
Epoch: 14 Loss: 0.15139408
Epoch: 15 Loss: 0.14566262
Epoch: 16 Loss: 0.14030810
Epoch: 17 Loss: 0.13544121
Epoch: 18 Loss: 0.13110325
Epoch: 19 Loss: 0.12727928
Epoch: 20 Loss: 0.12391797
Epoch: 21 Loss: 0.12095395
Epoch: 22 Loss: 0.11832570
Epoch: 23 Loss: 0.11598344
Epoch: 24 Loss: 0.11389031
Epoch: 25 Loss: 0.11202089
Epoch: 26 Loss: 0.11036178
Epoch: 27 Loss: 0.10891450
Epoch: 28 Loss: 0.10769790
Epoch: 29 Loss: 0.10674413
Epoch: 30 Loss: 0.10608405
Epoch: 31 Loss: 0.10575975
Epoch: 32 Loss: 0.10588431
Epoch: 33 Loss: 0.10653228
Epoch: 34 Loss: 0.10749187
Epoch: 35 Loss: 0.10844672
Epoch: 36 Loss: 0.10924806
Epoch: 37 Loss: 0.10987102
E

In [46]:
# DON'T WRITE HERE

### 9. Evaluate the model using the test set
Be sure to re-initialize the hidden parameters $h_0$ and $c_0$ before running the model.

In [47]:
future = 12

# Add the last window of training values to the list of predictions
preds = train_norm[-window_size:].tolist()

# Set the model to evaluation mode
model.eval()

for i in range(future):
    seq = torch.FloatTensor(preds[-window_size:])
    with torch.no_grad():
        model.hidden = (torch.zeros(1,1,model.hidden_size),
                        torch.zeros(1,1,model.hidden_size))
        preds.append(model(seq).item())



In [48]:
# Run the code below to check your results:
preds[window_size:]

[0.2540278434753418,
 -0.002775155007839203,
 -0.34294819831848145,
 -0.21167579293251038,
 0.23918873071670532,
 0.48914825916290283,
 0.24707987904548645,
 -0.08662215620279312,
 -0.25781765580177307,
 0.02258601039648056,
 0.5441880822181702,
 0.6110795736312866]

In [49]:
# DON'T WRITE HERE

## 10. Inverse transform the predicted values
Rescale the predicted values up to the original test set range.

In [50]:
# CODE HERE
true_predictions = scaler.inverse_transform(np.array(preds[window_size:]).reshape(-1, 1))
true_predictions
df['IPG2211A2N'][-12:]

313    107.6711
314    104.5716
315     93.4491
316     93.2405
317    103.3083
318    113.3178
319    113.2758
320    101.5656
321     95.3203
322    103.5750
323    112.9498
324    123.7687
Name: IPG2211A2N, dtype: float64

In [51]:
# Run the code below to check your results:
true_predictions

array([[105.95763959],
       [ 98.05722374],
       [ 87.59197015],
       [ 91.63050007],
       [105.50112171],
       [113.19100162],
       [105.74388894],
       [ 95.47771268],
       [ 90.21096873],
       [ 98.83744732],
       [114.88427426],
       [116.94215754]])

In [52]:
# DON'T WRITE HERE

## BONUS EXERCISE: Plot the result
Plot the true_predictions values together with the original data. Remember to create a range of datetime values for the predicted data.

In [53]:
# CODE HERE
# Remember that the stop date has to be later than the last predicted value.
x = np.arange('2016-02-01', '2019-02-01', dtype='datetime64[M]').astype('datetime64[D]')
x



array(['2016-02-01', '2016-03-01', '2016-04-01', '2016-05-01',
       '2016-06-01', '2016-07-01', '2016-08-01', '2016-09-01',
       '2016-10-01', '2016-11-01', '2016-12-01', '2017-01-01',
       '2017-02-01', '2017-03-01', '2017-04-01', '2017-05-01',
       '2017-06-01', '2017-07-01', '2017-08-01', '2017-09-01',
       '2017-10-01', '2017-11-01', '2017-12-01', '2018-01-01',
       '2018-02-01', '2018-03-01', '2018-04-01', '2018-05-01',
       '2018-06-01', '2018-07-01', '2018-08-01', '2018-09-01',
       '2018-10-01', '2018-11-01', '2018-12-01', '2019-01-01'],
      dtype='datetime64[D]')

In [54]:
plt.figure(figsize=(12,4))
plt.title('Energy')
plt.ylabel('Sales (millions of dollars)')
plt.grid(True)
plt.autoscale(axis='x',tight=True)
plt.plot(df['IPG2211A2N'])
plt.plot(x,true_predictions)
plt.show()

ValueError: ignored

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f6f17a2c830> (for post_execute):


ValueError: ignored

ValueError: ignored

In [ ]:
# DON'T WRITE HERE

In [ ]:
# DON'T WRITE HERE

## Great job!